In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
print(os.getcwd())
data_dir = "data"
data_frames = {}
for file_name in os.listdir(data_dir):
	file_path = os.path.join(data_dir, file_name)
	if file_name.endswith('.csv'):
		data_frames[file_name[:-4]] = pd.read_csv(file_path)
print("Loaded data frames:", list(data_frames.keys()))

print([data_frame.shape for data_frame in data_frames.values()])
def preprocess_data(data_frames):
    for name, df in data_frames.items():
        data_frames[name] = df.drop_duplicates()
    return data_frames
print([data_frame_name for data_frame_name in data_frames.keys()])

/home/wren/D/1S4/DSC/letterbox-project/phase2/scripts


FileNotFoundError: [Errno 2] No such file or directory: 'phase2/data'

In [ ]:
def normalize_watch_count(data_frames,table):	
	watch_counts=data_frames[table]['watch_count']
	log_watch_counts=np.log1p(watch_counts)
	scalar=MinMaxScaler()
	normalized_watch_counts=scalar.fit_transform(log_watch_counts.values.reshape(-1, 1))
	data_frames[table]['normalized_watch_count']=normalized_watch_counts



In [ ]:
normalize_watch_count(data_frames,'Director_count_df')
normalize_watch_count(data_frames,'Countries_count_df')
normalize_watch_count(data_frames,'Cast_count_df')
normalize_watch_count(data_frames,'Languages_count_df')
normalize_watch_count(data_frames,'decade_count_df')


In [ ]:
from sklearn.preprocessing import StandardScaler

def noramlize_ratings (data_frames,table,column):
	scaler = StandardScaler(with_std=True,with_mean=True)
	data_frames[table]['normalized_rating'] = scaler.fit_transform(data_frames[table]['rating'].values.reshape(-1, 1))


In [ ]:
noramlize_ratings(data_frames,'Director_rating_df','director_id')
noramlize_ratings(data_frames,'Countries_rating_df','director_id')
noramlize_ratings(data_frames,'Cast_rating_df','director_id')
noramlize_ratings(data_frames,'Language_rating_df','director_id')
noramlize_ratings(data_frames,'decade_rating_df','director_id')


In [ ]:
for name, df in data_frames.items():
	file_name = f"data/{name}.csv"
	df.to_csv(file_name, index=False)
	print(f"Saved {name} to {file_name}")


Saved Writer_df to ../data/Writer_df.csv
Saved Movie_Director_df to ../data/Movie_Director_df.csv
Saved decade_rating_df to ../data/decade_rating_df.csv
Saved Director_count_df to ../data/Director_count_df.csv
Saved Movie_Writer_df to ../data/Movie_Writer_df.csv
Saved Movie_Studio_df to ../data/Movie_Studio_df.csv
Saved Countries_rating_df to ../data/Countries_rating_df.csv
Saved Country_df to ../data/Country_df.csv
Saved Director_df to ../data/Director_df.csv
Saved Movie_Cast_df to ../data/Movie_Cast_df.csv
Saved Countries_count_df to ../data/Countries_count_df.csv
Saved User_df to ../data/User_df.csv
Saved Movie_Composer_df to ../data/Movie_Composer_df.csv
Saved Movie_Genres_df to ../data/Movie_Genres_df.csv
Saved Movie_Country_df to ../data/Movie_Country_df.csv
Saved Director_rating_df to ../data/Director_rating_df.csv
Saved Language_rating_df to ../data/Language_rating_df.csv
Saved Composer_df to ../data/Composer_df.csv
Saved Languages_count_df to ../data/Languages_count_df.csv
Sav

In [ ]:
import pandas as pd
import os

def generate_pivot_file(input_path,output_path,id_col,value_col,score_col,output_prefix,top_k,score_type):
    df = pd.read_csv(input_path)
    top_items = (
        df.sort_values([value_col, score_col], ascending=[True, False])
          .groupby(value_col)
          .head(top_k)
    )
    top_items['rank'] = top_items.groupby(value_col).cumcount() + 1
    item_pivot = top_items.pivot(index=value_col, columns='rank', values=id_col)
    item_pivot.columns = [f'{output_prefix}{i}_{score_type}' for i in item_pivot.columns]
    score_pivot = top_items.pivot(index=value_col, columns='rank', values=score_col)
    score_pivot.columns = [f'{output_prefix}{i}_norm_{score_type}' for i in score_pivot.columns]
    final_df = pd.concat([item_pivot, score_pivot], axis=1).reset_index()
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    final_df.to_csv(output_path, index=False)
    print(f"{output_path} created successfully.")

In [ ]:
# ----------- Casts -----------
generate_pivot_file(
    input_path="data/Cast_count_df.csv",
    output_path="data_pivot/Cast_count_df_pivot.csv",
    id_col='cast_id',
    value_col='user_id',
    score_col='normalized_watch_count',
    output_prefix='cast',
    top_k = 10,
    score_type='count'
)

generate_pivot_file(
    input_path="data/Cast_rating_df.csv",
    output_path="data_pivot/Cast_rating_df_pivot.csv",
    id_col='cast_id',
    value_col='user_id',
    score_col='normalized_rating',
    output_prefix='cast',
    top_k = 10,
    score_type='rate'
)

# ----------- Countries -----------
generate_pivot_file(
    input_path="data/Countries_count_df.csv",
    output_path="data_pivot/Countries_count_df_pivot.csv",
    id_col='country_id',
    value_col='user_id',
    score_col='normalized_watch_count',
    output_prefix='country',
    top_k = 10,
    score_type='count'
)

generate_pivot_file(
    input_path="data/Countries_rating_df.csv",
    output_path="data_pivot/Countries_rating_df_pivot.csv",
    id_col='country_id',
    value_col='user_id',
    score_col='normalized_rating',
    output_prefix='country',
    top_k = 10,
    score_type='rate'
)

# ----------- Directors -----------
generate_pivot_file(
    input_path="data/Director_count_df.csv",
    output_path="data_pivot/Director_count_df_pivot.csv",
    id_col='director_id',
    value_col='user_id',
    score_col='normalized_watch_count',
    output_prefix='director',
    top_k = 10,
    score_type='count'
)

generate_pivot_file(
    input_path="data/Director_rating_df.csv",
    output_path="data_pivot/Director_rating_df_pivot.csv",
    id_col='director_id',
    value_col='user_id',
    score_col='normalized_rating',
    output_prefix='director',
    top_k = 10,
    score_type='rate'
)

# ----------- Languages -----------
generate_pivot_file(
    input_path="data/Languages_count_df.csv",
    output_path="data_pivot/Language_count_df_pivot.csv",
    id_col='language_id',
    value_col='user_id',
    score_col='normalized_watch_count',
    output_prefix='lang',
    top_k = 10,
    score_type='count'
)

generate_pivot_file(
    input_path="data/Language_rating_df.csv",
    output_path="data_pivot/Language_rating_df_pivot.csv",
    id_col='language_id',
    value_col='user_id',
    score_col='normalized_rating',
    output_prefix='lang',
    top_k = 3,
    score_type='rate'
)

# ----------- Decades -----------
generate_pivot_file(
    input_path="data/decade_count_df.csv",
    output_path="data_pivot/Decade_count_df_pivot.csv",
    id_col='decade',
    value_col='user_id',
    score_col='normalized_watch_count',
    output_prefix='decade',
    top_k = 8,
    score_type='count'
)

generate_pivot_file(
    input_path="data/decade_rating_df.csv",
    output_path="data_pivot/Decade_rating_df_pivot.csv",
    id_col='decade',
    value_col='user_id',
    score_col='normalized_rating',
    output_prefix='decade',
    top_k = 7,
    score_type='rate'
)


In [ ]:
import pandas as pd
import os

file_paths = [
    "data_pivot/Cast_count_df_pivot.csv",
    "data_pivot/Cast_rating_df_pivot.csv",
    "data_pivot/Countries_count_df_pivot.csv",
    "data_pivot/Countries_rating_df_pivot.csv",
    "data_pivot/Decade_count_df_pivot.csv",
    "data_pivot/Decade_rating_df_pivot.csv",
    "data_pivot/Director_count_df_pivot.csv",
    "data_pivot/Director_rating_df_pivot.csv",
    "data_pivot/Language_count_df_pivot.csv",
    "data_pivot/Language_rating_df_pivot.csv",
    "data/User_Favorite_Genre_df.csv",    
]

existing_files = [f for f in file_paths if os.path.exists(f)]

final_df = pd.read_csv(existing_files[0])
for path in existing_files[1:]:
    df = pd.read_csv(path)
    final_df = pd.merge(final_df, df, on='user_id', how='inner')
final_df.to_csv("data_pivot/ALL_features_merged.csv", index=False)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

def clean_data(df):
    # Replace missing values with the mean of each column in: 'rating_std'
    df = df.fillna({'rating_std': df['rating_std'].mean()})
    # Replace missing values with 0 in column: 'actor_5_id'
    df = df.fillna({'actor_5_id': 0})
    # Replace missing values with 0 in column: 'actor_4_id'
    df = df.fillna({'actor_4_id': 0})
    # Replace missing values with 0 in column: 'actor_3_id'
    df = df.fillna({'actor_3_id': 0})
    # Replace missing values with 0 in column: 'actor_2_id'
    df = df.fillna({'actor_2_id': 0})
    # Replace missing values with 0 in column: 'actor_1_id'
    df = df.fillna({'actor_1_id': 0})
    # Replace missing values with 0 in column: 'composer_id'
    df = df.fillna({'composer_id': 0})
    # Replace all instances of 0 with 358 in column: 'composer_id'
    df.loc[df['composer_id'] == 0, 'composer_id'] = 358
    # Replace all instances of 0 with 4868 in column: 'actor_1_id'
    df.loc[df['actor_1_id'] == 0, 'actor_1_id'] = 4868
    # Replace all instances of 0 with 4868 in column: 'actor_2_id'
    df.loc[df['actor_2_id'] == 0, 'actor_2_id'] = 4868
    # Replace all instances of 0 with 4868 in column: 'actor_3_id'
    df.loc[df['actor_3_id'] == 0, 'actor_3_id'] = 4868
    # Replace all instances of 0 with 4868 in column: 'actor_4_id'
    df.loc[df['actor_4_id'] == 0, 'actor_4_id'] = 4868
    # Replace all instances of 0 with 4868 in column: 'actor_5_id'
    df.loc[df['actor_5_id'] == 0, 'actor_5_id'] = 4868
    # Replace missing values with 566 in column: 'writer_id'
    df = df.fillna({'writer_id': 566})
    # Scale column 'average_rating' between -1 and 1

    # Scale column 'duration' between 0 and 1
    
    scaler = MinMaxScaler()
    df['year'] = scaler.fit_transform(df[['year']])
    scaler = MinMaxScaler()
    df['duration'] = scaler.fit_transform(df[['duration']])
    scaler = MinMaxScaler()
    df['average_rating'] = scaler.fit_transform(df[['average_rating']])
    return df

df = pd.read_csv('data/Final_Movie_Data_df.csv')

df_clean = clean_data(df.copy())
df_clean.to_csv('data/Final_Movie_Data.csv', index=False)
df_clean.head()